In [ ]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

## API Overview

This notebook provides a demonstration of how to interact with graphrag as a library using the API as opposed to the CLI. Note that graphrag's CLI actually connects to the library through this API for all operations. 

In [ ]:
import graphrag.api as api
from graphrag.index.typing import PipelineRunResult

## Prerequisite
As a prerequisite to all API operations, a `GraphRagConfig` object is required. It is the primary means to control the behavior of graphrag and can be instantiated from a `settings.yaml` configuration file.

Please refer to the [CLI docs](https://microsoft.github.io/graphrag/cli/#init) for more detailed information on how to generate the `settings.yaml` file.

#### Load `settings.yaml` configuration

In [ ]:
import yaml

settings = yaml.safe_load(open("<project_directory>/settings.yaml"))  # noqa: PTH123, SIM115

At this point, you can modify the imported settings to align with your application's requirements. For example, if building a UI application, the application might need to change the input and/or storage destinations dynamically in order to enable users to build and query different indexes.

### Generate a `GraphRagConfig` object

In [ ]:
from graphrag.config.create_graphrag_config import create_graphrag_config

graphrag_config = create_graphrag_config(
    values=settings, root_dir="<project_directory>"
)

## Indexing API

*Indexing* is the process of ingesting raw text data and constructing a knowledge graph. GraphRAG currently supports plaintext (`.txt`) and `.csv` file formats.

## Build an index

In [ ]:
index_result: list[PipelineRunResult] = await api.build_index(config=graphrag_config)

# index_result is a list of workflows that make up the indexing pipeline that was run
for workflow_result in index_result:
    status = f"error\n{workflow_result.errors}" if workflow_result.errors else "success"
    print(f"Workflow Name: {workflow_result.workflow}\tStatus: {status}")

## Query an index

To query an index, several index files must first be read into memory and passed to the query API. 

In [ ]:
import pandas as pd

final_nodes = pd.read_parquet("<project_directory>/output/create_final_nodes.parquet")
final_entities = pd.read_parquet(
    "<project_directory>/output/create_final_entities.parquet"
)
final_communities = pd.read_parquet(
    "<project_directory>/output/create_final_communities.parquet"
)
final_community_reports = pd.read_parquet(
    "<project_directory>/output/create_final_community_reports.parquet"
)

response, context = await api.global_search(
    config=graphrag_config,
    nodes=final_nodes,
    entities=final_entities,
    communities=final_communities,
    community_reports=final_community_reports,
    community_level=2,
    dynamic_community_selection=False,
    response_type="Multiple Paragraphs",
    query="Who is Scrooge and what are his main relationships?",
)

The response object is the official reponse from graphrag while the context object holds various metadata regarding the querying process used to obtain the final response.

In [ ]:
print(response)

Digging into the context a bit more provides users with extremely granular information such as what sources of data (down to the level of text chunks) were ultimately retrieved and used as part of the context sent to the LLM model).

In [ ]:
from pprint import pprint

pprint(context)  # noqa: T203